In [29]:
import numpy as np
import numpy.typing as npt
import matplotlib.pyplot as plt  # Biblioteca para gerar gráficos
import pandas as pd
from sklearn import metrics, model_selection
from scipy import stats
from scipy.spatial import distance
import math


In [30]:

class Set:
    def __init__(self, dataset, features, output):
        self.dataset = dataset
        self.features = features
        self.output = output

    def get_n(self):
        return self.dataset.shape[0]

    def get_x(self):
        return self.dataset[:, self.features]
    
    def get_x_apply(self, func):
        return func(self.dataset[:, self.features])

    def set_x(self, new_x):
        self.x = new_x

    def get_y(self):
        return self.dataset[:, self.output]

    def get_X(self, func=None):
        if (func):
            return np.c_[np.ones(self.get_n()), func(self.get_x())]
        else:
            return np.c_[np.ones(self.get_n()), self.get_x()]


In [31]:
def is_true_positive(y, y_pred):
    return y_pred >= 1 and y >= 1

def is_false_positive(y, y_pred):
    return y_pred >= 1 and y <= 0

def is_true_negative(y, y_pred):
    return y_pred <= 0 and y <= 0

def is_false_negative(y, y_pred):
    return y_pred <= 0 and y >= 1

def confusion_matrix(y, y_pred):
    """ returns (tp, fp, tn, fn) """

    tp, fp, tn, fn = 0, 0, 0, 0
    for i, pred in enumerate(y_pred):
        tp += 1 if is_true_positive(y[i], pred) else 0
        fp += 1 if is_false_positive(y[i], pred) else 0
        tn += 1 if is_true_negative(y[i], pred) else 0
        fn += 1 if is_false_negative(y[i], pred) else 0
    return (tp, fp, tn, fn)

def accuracy(y, y_pred):
    tp, fp, tn, fn = confusion_matrix(y, y_pred)
    return (tp + tn) / (tp + fp + tn + fn)

def precision(y, y_pred):
    tp, fp, tn, fn = confusion_matrix(y, y_pred)
    return tp / (tp + fp)

def recall(y, y_pred):
    tp, fp, tn, fn = confusion_matrix(y, y_pred)
    return tp / (tp + fn)

def f1_score(y, y_pred):
    precision_ = precision(y, y_pred)
    recall_ = recall(y, y_pred)
    return 2 * (precision_ * recall_) / (precision_ + recall_)

def get_metrics(n_folds):
    return {
        "accuracy": np.zeros(n_folds),
        "precision": np.zeros(n_folds),
        "recall": np.zeros(n_folds),
        "f1_score": np.zeros(n_folds)
    }

def calculate_metrics(metrics, y_test, y_pred):
    metrics["accuracy"][i] = (accuracy(y_test, y_pred))
    metrics["precision"][i] = (precision(y_test, y_pred))
    metrics["recall"][i] = (recall(y_test, y_pred))
    metrics["f1_score"][i] = (f1_score(y_test, y_pred))

def print_metrics(metrics, name, n_folds):
    print("%i-fold cross validation com %s" % (n_folds, name))
    print("acurácia: %.8f +/- %.8f" % (metrics["accuracy"].mean(), metrics["accuracy"].std()))
    print("revocação: %.8f +/- %.8f" % (metrics["precision"].mean(), metrics["precision"].std()))
    print("precisão: %.8f +/- %.8f" % (metrics["recall"].mean(), metrics["recall"].std()))
    print("f1-score: %.8f +/- %.8f" % (metrics["f1_score"].mean(), metrics["f1_score"].std()))
    print("")


In [32]:
def k_fold_split(array, k = int):
    """realiza o split dos dados em k-folds"""
    shuffled_data = np.random.permutation(array)
    folds = np.array_split(shuffled_data, k)
    return folds

def k_fold_train_test(folds):
    """retorna um vetor com as configurações de treino e teste definidas pelo k-fold split

    returns (i, train, test)
    """
    results = []
    for i, fold in enumerate(folds):
        train = np.vstack([x for j, x in enumerate(folds) if j != i])
        test = fold
        results.append((i, train, test))
    return results

# Questão 1

Considere o conjunto de dados disponível em **concrete.csv**, organizado em 9 colunas, sendo as 8 primeiras colunas os atributos e a última coluna a saída. Os 8 atributos referem-se à caracterização de diferentes tipos de concreto para construção civil. A saída é a resistência à compressão do concreto (em megapascals, MPa). Maiores detalhes sobre os dados podem ser conferidos em https://www.openml.org/d/4353.

In [33]:
data = np.genfromtxt('concrete.csv', delimiter=',')
np.random.seed(666)
features = np.arange(8)
output = 8

a) Considere um modelo de regressão não linear baseado em redes neurais artificiais. Separe os dados aleatoriamente em treino, validação e teste (por exemplo, 60%, 20% e 20%). Nesse cenário, treine e avalie o modelo abaixo:

- **MLP (multilayer perceptron)**: 1 camada oculta e treinamento em *minibatch* via gradiente descendente estocástico com termo de *momentum*. Utilize o conjunto de validação para a justar os hiperparâmetros.

In [34]:
data = Set(data, features, output)
train_X, test_validation_X, train_y, test_validation_y = model_selection.train_test_split(data.get_X(func=stats.zscore), data.get_y(), test_size=0.4, random_state=666, shuffle=True)
test_X, validation_X, test_y, validation_y = model_selection.train_test_split(test_validation_X, test_validation_y, test_size=0.5, random_state=666)

def accuracy_score(y_true, y_pred):
    """ Compare y_true to y_pred and return the accuracy """
    accuracy = np.sum(y_true == y_pred, axis=0) / len(y_true)
    return accuracy

class TanH():
    def __call__(self, x):
        return 2 / (1 + np.exp(-2*x)) - 1

    def gradient(self, x):
        return 1 - np.power(self.__call__(x), 2)

class ReLU():
    def __call__(self, x):
        return np.where(x >= 0, x, 0)

    def gradient(self, x):
        return np.where(x >= 0, 1, 0)

class Loss(object):
    def loss(self, y_true, y_pred):
        return NotImplementedError()

    def gradient(self, y, y_pred):
        raise NotImplementedError()

    def acc(self, y, y_pred):
        return 0

class CrossEntropy(Loss):
    def __init__(self): pass

    def loss(self, y, p):
        # Avoid division by zero
        p = np.clip(p, 1e-15, 1 - 1e-15)
        return - y * np.log(p) - (1 - y) * np.log(1 - p)

    def acc(self, y, p):
        return accuracy_score(np.argmax(y, axis=1), np.argmax(p, axis=1))

    def gradient(self, y, p):
        # Avoid division by zero
        p = np.clip(p, 1e-15, 1 - 1e-15)
        return - (y / p) + (1 - y) / (1 - p)

class Softmax():
    def __call__(self, x):
        e_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
        return e_x / np.sum(e_x, axis=-1, keepdims=True)

    def gradient(self, x):
        p = self.__call__(x)
        return p * (1 - p)

class Sigmoid():
    def __call__(self, x):
        return 1 / (1 + np.exp(-x))

    def gradient(self, x):
        return self.__call__(x) * (1 - self.__call__(x))

class MultilayerPerceptron():
    """Multilayer Perceptron classifier. A fully-connected neural network with one hidden layer.
    Unrolled to display the whole forward and backward pass.
    Parameters:
    -----------
    n_hidden: int:
        The number of processing nodes (neurons) in the hidden layer. 
    n_iterations: float
        The number of training iterations the algorithm will tune the weights for.
    learning_rate: float
        The step length that will be used when updating the weights.
    """
    def __init__(self, n_hidden, n_iterations=3000, learning_rate=0.01):
        self.n_hidden = n_hidden
        self.n_iterations = n_iterations
        self.learning_rate = learning_rate
        self.hidden_activation = Sigmoid()
        self.output_activation = Softmax()
        self.loss = CrossEntropy()

    def _initialize_weights(self, X, y):
        n_samples, n_features = X.shape
        _, n_outputs = 1, y.shape[0]

        # Hidden layer
        limit   = 1 / math.sqrt(n_features)
        self.W  = np.random.uniform(-limit, limit, (n_features, self.n_hidden))
        self.w0 = np.zeros((1, self.n_hidden))
        # Output layer
        limit   = 1 / math.sqrt(self.n_hidden)
        self.V  = np.random.uniform(-limit, limit, (self.n_hidden, n_outputs))
        self.v0 = np.zeros((1, n_outputs))

    def fit(self, X, y):

        self._initialize_weights(X, y)

        for i in range(self.n_iterations):

            # ..............
            #  Forward Pass
            # ..............

            # HIDDEN LAYER
            hidden_input = X.dot(self.W) + self.w0
            hidden_output = self.hidden_activation(hidden_input)
            # OUTPUT LAYER
            output_layer_input = hidden_output.dot(self.V) + self.v0
            y_pred = self.output_activation(output_layer_input)

            # ...............
            #  Backward Pass
            # ...............

            # OUTPUT LAYER
            # Grad. w.r.t input of output layer
            grad_wrt_out_l_input = self.loss.gradient(y, y_pred) * self.output_activation.gradient(output_layer_input)
            grad_v = hidden_output.T.dot(grad_wrt_out_l_input)
            grad_v0 = np.sum(grad_wrt_out_l_input, axis=0, keepdims=True)
            # HIDDEN LAYER
            # Grad. w.r.t input of hidden layer
            grad_wrt_hidden_l_input = grad_wrt_out_l_input.dot(self.V.T) * self.hidden_activation.gradient(hidden_input)
            grad_w = X.T.dot(grad_wrt_hidden_l_input)
            grad_w0 = np.sum(grad_wrt_hidden_l_input, axis=0, keepdims=True)

            # Update weights (by gradient descent)
            # Move against the gradient to minimize loss
            self.V  -= self.learning_rate * grad_v
            self.v0 -= self.learning_rate * grad_v0
            self.W  -= self.learning_rate * grad_w
            self.w0 -= self.learning_rate * grad_w0

    # Use the trained model to predict labels of X
    def predict(self, X):
        # Forward pass:
        hidden_input = X.dot(self.W) + self.w0
        hidden_output = self.hidden_activation(hidden_input)
        output_layer_input = hidden_output.dot(self.V) + self.v0
        y_pred = self.output_activation(output_layer_input)
        return y_pred

mlp = MultilayerPerceptron(2, 100, 0.01)
mlp.fit(train_X, train_y)
y_pred = mlp.predict(validation_X)

print(validation_X.shape)
print(validation_y.shape)
print(y_pred.shape)
accuracy_score(validation_y, y_pred)

(206, 9)
(206,)
(206, 618)


C:\Users\rflav\AppData\Local\Temp/ipykernel_19384/336640175.py:7: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  accuracy = np.sum(y_true == y_pred, axis=0) / len(y_true)


0.0

b) Apresente as curvas da função custo nos conjuntos de treinamento e validação ao longo das épocas. Reporte também para os conjuntos de treino, validação e teste as métricas abaixo:
- **RMSE (root mean squared error)**: 
- **MAE (mean absolute error)**:
- **MRE (mean relative error)**:


## Questão 2

Considere o conjunto de dados disponível em **vowel.csv**, organizado em 11 colunas, sendo as 10 primeiras colunas os atributos e a última coluna a saída. Os 10 atributos referem-se à caracterização de amostras da fala de britânicos. A saída é o fonema de vogal correspondente, dentre as 11 possibilidades. Maiores detalhes sobre os dados podem ser conferidos em https://www.openml.org/d/307.

In [35]:
data = np.genfromtxt('vowel.csv', delimiter=',')
np.random.seed(666)
features = np.arange(11)
output = 11

a) Considere um mo delo de classificação não linear baseado em redes neurais artificiais. Separe os dados aleatoriamente em treino, validação e teste (por exemplo, 60%, 20% e 20%). Nesse cenário, treine e avalie o modelo abaixo:

- **MLP (multilayer perceptron)**: 1 camada o culta e treinamento em *minibatch* via gradiente descendente esto cástico com termo de *momentum*. Utilize o conjunto de validação para a justar os hiperparâmetros.

b) Apresente as curvas da função custo nos conjuntos de treinamento e validação ao longo das épocas. Reporte também a acurácia obtida para os conjuntos de treino, validação e teste.